# Cross-check .nc file differences between input directories

Set `old` and `new` as appropriate.

In [1]:
import dask.array as da
import xarray as xr
import numpy as np
import glob
import os.path

In [2]:
import climtas.nci
climtas.nci.GadiClient(malloc_trim_threshold='64kib')

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 27
Total threads: 27,Total memory: 192.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38963,Workers: 27
Dashboard: /proxy/8787/status,Total threads: 27
Started: Just now,Total memory: 192.00 GiB
Comm: tcp://127.0.0.1:38057,Total threads: 1
Dashboard: /proxy/46067/status,Memory: 7.11 GiB
Nanny: tcp://127.0.0.1:38135,


In [3]:
old = '/g/data/ik11/inputs/access-om2/input_bgc_20211126/'
new = '/g/data/ik11/inputs/access-om2/input_bgc_20220224/'
ofiles = [ os.path.relpath(fpath, old) for fpath in glob.glob(old+'*/*.nc') if 'CHUCKABLE' not in fpath]
ofiles.sort()
nfiles = [ os.path.relpath(fpath, new) for fpath in glob.glob(new+'*/*.nc') if 'CHUCKABLE' not in fpath ]
nfiles.sort()

In [4]:
ofiles

['cice_01deg-cycle4-iced/iced.2019-01-01-00000.nc',
 'cice_01deg-cycle4/i2o.nc',
 'cice_01deg-cycle4/o2i.nc',
 'cice_01deg/i2o.nc',
 'cice_01deg/o2i.nc',
 'cice_025deg/i2o.nc',
 'cice_025deg/o2i.nc',
 'cice_1deg/i2o.nc',
 'cice_1deg/o2i.nc',
 'mom_01deg-cycle4/csiro_bgc.res.nc',
 'mom_01deg-cycle4/ocean_sbc.res.nc',
 'mom_01deg/bgc_param.nc',
 'mom_01deg/co2_iaf.nc',
 'mom_01deg/co2_ryf.nc',
 'mom_01deg/csiro_bgc.res.nc',
 'mom_01deg/csiro_bgc_sediment.res.nc',
 'mom_01deg/dust.nc',
 'mom_025deg/bgc_param.nc',
 'mom_025deg/co2_iaf.nc',
 'mom_025deg/co2_ryf.nc',
 'mom_025deg/csiro_bgc.res.nc',
 'mom_025deg/csiro_bgc_sediment.res.nc',
 'mom_025deg/dust.nc',
 'mom_1deg/bgc_param.nc',
 'mom_1deg/co2_iaf.nc',
 'mom_1deg/co2_ryf.nc',
 'mom_1deg/csiro_bgc.res.nc',
 'mom_1deg/csiro_bgc_sediment.res.nc',
 'mom_1deg/dust.nc']

In [5]:
nfiles

['cice_01deg-cycle4-iced/iced.2019-01-01-00000.nc',
 'cice_01deg-cycle4/i2o.nc',
 'cice_01deg-cycle4/o2i.nc',
 'cice_01deg/i2o.nc',
 'cice_01deg/o2i.nc',
 'cice_025deg/i2o.nc',
 'cice_025deg/o2i.nc',
 'cice_1deg/i2o.nc',
 'cice_1deg/o2i.nc',
 'mom_01deg-cycle4/csiro_bgc.res.nc',
 'mom_01deg-cycle4/ocean_sbc.res.nc',
 'mom_01deg/bgc_param.nc',
 'mom_01deg/co2_iaf.nc',
 'mom_01deg/co2_ryf.nc',
 'mom_01deg/csiro_bgc.res.nc',
 'mom_01deg/csiro_bgc_sediment.res.nc',
 'mom_01deg/dust.nc',
 'mom_025deg/bgc_param.nc',
 'mom_025deg/co2_iaf.nc',
 'mom_025deg/co2_ryf.nc',
 'mom_025deg/csiro_bgc.res.nc',
 'mom_025deg/csiro_bgc_sediment.res.nc',
 'mom_025deg/dust.nc',
 'mom_1deg/bgc_param.nc',
 'mom_1deg/co2_iaf.nc',
 'mom_1deg/co2_ryf.nc',
 'mom_1deg/csiro_bgc.res.nc',
 'mom_1deg/csiro_bgc_sediment.res.nc',
 'mom_1deg/dust.nc']

In [6]:
if ofiles != nfiles:
    print('ERROR: old and new file lists differ')
else:
    for f in ofiles:
        print(f, end=' ')
        dsold = xr.open_dataset(old+f, decode_times=False)
        dsnew = xr.open_dataset(new+f, decode_times=False)
        if dsold.identical(dsnew):
            print(True, True)
        else:
            if dsold.equals(dsnew):
                print(False, True)
            else:
                print(False, False)
                ovars = set(dsold.keys())
                nvars = set(dsnew.keys())
                if ovars != nvars:
                    print('  NOTE: variable names differ - will only check the common variables')
                    print('  ovars:', ovars)
                    print('  nvars:', nvars)
                vs = list(ovars & nvars)
                vs.sort()
                for v in vs:
                    idcl = dsold[v].identical(dsnew[v])
                    if not idcl:
                        eql = dsold[v].equals(dsnew[v])
                    else:
                        eql = True  # identical implies equal
                    if eql:
                        print('    ', v, idcl, eql)
                    else:
                        try:
                            ratio = dsnew[v].data/dsold[v].data
                            diff = dsnew[v].data - dsold[v].data
                            print('    ', v, idcl, eql, np.nanmin(ratio), np.nanmax(ratio), np.nanmin(diff), np.nanmax(diff))
                        except:
                            print('    ', v, idcl, eql)

cice_01deg-cycle4-iced/iced.2019-01-01-00000.nc False True
cice_01deg-cycle4/i2o.nc False True
cice_01deg-cycle4/o2i.nc False True
cice_01deg/i2o.nc False True
cice_01deg/o2i.nc False True
cice_025deg/i2o.nc False True
cice_025deg/o2i.nc False True
cice_1deg/i2o.nc False True
cice_1deg/o2i.nc False True
mom_01deg-cycle4/csiro_bgc.res.nc False False
  NOTE: variable names differ - will only check the common variables
  ovars: {'det', 'salt', 'caco3', 'fe', 'alk', 'adic', 'o2', 'dic', 'zoo', 'phy', 'no3'}
  nvars: {'det', 'caco3', 'fe', 'alk', 'no3', 'o2', 'dic', 'zoo', 'phy', 'adic'}
     adic False True
     alk False True
     caco3 False True
     det False True
     dic False True
     fe False True
     no3 False True


/jobfs/35975466.gadi-pbs/ipykernel_1016724/953087186.py:33: RuntimeWarning: invalid value encountered in true_divide
  ratio = dsnew[v].data/dsold[v].data


     o2 False False 0.022950672846555154 0.02295067284655516 -19916.989432471946 0.0
     phy False True
     zoo False True
mom_01deg-cycle4/ocean_sbc.res.nc True True
mom_01deg/bgc_param.nc False True
mom_01deg/co2_iaf.nc False True
mom_01deg/co2_ryf.nc False True
mom_01deg/csiro_bgc.res.nc False False
  NOTE: variable names differ - will only check the common variables
  ovars: {'det', 'salt', 'caco3', 'fe', 'alk', 'adic', 'o2', 'dic', 'zoo', 'phy', 'no3'}
  nvars: {'det', 'caco3', 'fe', 'alk', 'no3', 'o2', 'dic', 'zoo', 'phy', 'adic'}
     adic False True
     alk False True
     caco3 False True
     det False True
     dic False True
     fe False True
     no3 False True


/jobfs/35975466.gadi-pbs/ipykernel_1016724/953087186.py:33: RuntimeWarning: invalid value encountered in true_divide
  ratio = dsnew[v].data/dsold[v].data


     o2 False False 0.022950672846555154 0.02295067284655516 -19916.989432471946 0.0
     phy False True
     zoo False True
mom_01deg/csiro_bgc_sediment.res.nc False True
mom_01deg/dust.nc False True
mom_025deg/bgc_param.nc False True
mom_025deg/co2_iaf.nc False True
mom_025deg/co2_ryf.nc False True
mom_025deg/csiro_bgc.res.nc False False
  NOTE: variable names differ - will only check the common variables
  ovars: {'det', 'salt', 'caco3', 'fe', 'alk', 'adic', 'o2', 'dic', 'zoo', 'phy', 'no3'}
  nvars: {'det', 'caco3', 'fe', 'alk', 'no3', 'o2', 'dic', 'zoo', 'phy', 'adic'}
     adic False True
     alk False True
     caco3 False True
     det False True
     dic False True
     fe False True
     no3 False True


/jobfs/35975466.gadi-pbs/ipykernel_1016724/953087186.py:33: RuntimeWarning: invalid value encountered in true_divide
  ratio = dsnew[v].data/dsold[v].data


     o2 False False 0.022950672846555154 0.02295067284655516 -19876.695509640307 0.0
     phy False True
     zoo False True
mom_025deg/csiro_bgc_sediment.res.nc False True
mom_025deg/dust.nc False True
mom_1deg/bgc_param.nc False True
mom_1deg/co2_iaf.nc False True
mom_1deg/co2_ryf.nc False True
mom_1deg/csiro_bgc.res.nc False False
  NOTE: variable names differ - will only check the common variables
  ovars: {'det', 'salt', 'caco3', 'fe', 'alk', 'adic', 'o2', 'dic', 'zoo', 'phy', 'no3'}
  nvars: {'det', 'caco3', 'fe', 'alk', 'no3', 'o2', 'dic', 'zoo', 'phy', 'adic'}
     adic False True
     alk False True
     caco3 False True
     det False True
     dic False True
     fe False True
     no3 False True


/jobfs/35975466.gadi-pbs/ipykernel_1016724/953087186.py:33: RuntimeWarning: invalid value encountered in true_divide
  ratio = dsnew[v].data/dsold[v].data


     o2 False False 0.022950672846555154 0.02295067284655516 -19834.728368099935 0.0
     phy False True
     zoo False True
mom_1deg/csiro_bgc_sediment.res.nc False True
mom_1deg/dust.nc False True
